In [ ]:
# Load 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import argparse, time, os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import InterpolationMode

import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

# Load 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import argparse, time, os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import InterpolationMode


# Define preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model input size
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Define the dataset

import numpy as np
import torch
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms
from sklearn.model_selection import StratifiedShuffleSplit

seed = 42
root = "/home/mrahm326/experiments/Probing Mechanism/figrim/Targets"

# Your existing train transform (from your snippet)
train_transform = transform

# Simple eval transform (adjust to your preprocessing: size/normalize, etc.)
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load once without transforms to read labels for a stratified split
full_base = datasets.ImageFolder(root=root)
y = np.array(full_base.targets)
idx = np.arange(len(y))

# Stratified split (keeps class balance)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
train_idx, test_idx = next(sss.split(idx, y))

# Build separate datasets so train/test can have different transforms
train_dataset = Subset(datasets.ImageFolder(root=root, transform=train_transform), train_idx.tolist())
test_dataset  = Subset(datasets.ImageFolder(root=root, transform=eval_transform),  test_idx.tolist())

# (Optional) DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,  num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import efficientnet_b0
from torchvision import transforms

num_classes = len(full_base.classes)
print(f"Found {num_classes} classes · {len(train_dataset)} train images · {len(test_dataset)} val images")


import torch

# Set device first
device = "cuda:1"

# Assume adv_patch is a tensor
def create_patch(patch_size=0.2, image_size=224):
    patch_dim = int(patch_size * image_size)
    patch = torch.rand(3, patch_dim, patch_dim, requires_grad=True, device=device)
    return patch
#torch.save(adv_patch, 'adversarial_patch.pth')
adv_patch = create_patch(patch_size=0.2, image_size=224)
import torch

# Load the saved patch
adv_patch = torch.load('/home/mrahm326/experiments/Probing Mechanism/DeepGaze/Figrim_ALL/adversarial_patch_figrim_eff.pth', map_location=device)

# If you need it to require gradients for optimization or testing:
adv_patch.requires_grad = True

# Confirm shape and device
print(f"Loaded patch shape: {adv_patch.shape}, requires_grad: {adv_patch.requires_grad}, device: {adv_patch.device}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sal_model = models.segmentation.deeplabv3_resnet50(weights=None, num_classes=1)  # Single channel output

from torchvision import models

# Load pretrained model
sal_model = models.segmentation.deeplabv3_resnet50(weights=models.segmentation.DeepLabV3_ResNet50_Weights.DEFAULT)

# Replace the classifier to output a single-channel saliency map
sal_model.classifier[-1] = torch.nn.Conv2d(256, 1, kernel_size=1)
# Load the best trained model
sal_model.load_state_dict(torch.load("/home/mrahm326/experiments/Probing Mechanism/DeepGaze/Figrim_ALL/best_deeplabv3_saliency_figrim.pt"))
sal_model.eval()

# Pretrained EfficientNet-B0
model_eff = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Replace the classifier head for your number of classes
in_features = model_eff.classifier[1].in_features  # linear layer is at index 1
model_eff.classifier[1] = nn.Linear(in_features, num_classes)

model_eff = model_eff.to(device)
model = model_eff 

model.load_state_dict(torch.load("best_eff_figrim.pt"))
model.to(device)
model.eval()

### Dataset for Adversarial Patches 

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
import os

DEVICE = "cuda:1"

# ==== Patch Application Function ====
def apply_patch(image_tensor, patch_tensor, device):
    patch_tensor = patch_tensor.to(device)
    patched_image = image_tensor.clone()
    _, H, W = image_tensor.shape
    ph, pw = patch_tensor.shape[1:]
    top = torch.randint(0, H - ph + 1, (1,)).item()
    left = torch.randint(0, W - pw + 1, (1,)).item()
    patched_image[:, top:top+ph, left:left+pw] = patch_tensor
    return patched_image

def apply_patch(image_tensor, patch_tensor, device):
    patch_tensor = patch_tensor.to(device)
    if patch_tensor.dim() == 4:  # remove batch dimension if present
        patch_tensor = patch_tensor.squeeze(0)
    patched_image = image_tensor.clone()
    _, H, W = image_tensor.shape
    _, ph, pw = patch_tensor.shape
    top = torch.randint(0, H - ph + 1, (1,)).item()
    left = torch.randint(0, W - pw + 1, (1,)).item()
    patched_image[:, top:top+ph, left:left+pw] = patch_tensor
    return patched_image


# ==== Custom Dataset with dual labels ====
class PatchDetectionDataset(Dataset):
    def __init__(self, root_dir, patch_tensor, transform=None, device="cpu"):
        self.root_dir = root_dir
        self.transform = transform
        self.patch = patch_tensor
        self.device = device

        self.image_paths = []   # (path, class_label, adv_flag)
        self.class_to_idx = {cls: i for i, cls in enumerate(sorted(os.listdir(root_dir)))}

        for class_name, class_idx in self.class_to_idx.items():
            class_path = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_path):
                continue

            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)

                if os.path.isdir(img_path):
                    continue
                if not img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
                    continue

                # Add both clean and adversarial versions
                self.image_paths.append((img_path, class_idx, 0))  # clean
                self.image_paths.append((img_path, class_idx, 1))  # patched

        # Expose labels for stratified splitting
        self.targets = [adv_flag for _, _, adv_flag in self.image_paths]
        self.class_labels = [class_idx for _, class_idx, _ in self.image_paths]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path, class_label, adv_flag = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        image_tensor = self.transform(image) if self.transform else transforms.ToTensor()(image)

        if adv_flag == 1:
            image_tensor = apply_patch(image_tensor, self.patch, self.device)

        # Return both labels
        return image_tensor, class_label, adv_flag


from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

# Create dataset
train_dataset = PatchDetectionDataset(
    root_dir="/home/mrahm326/experiments/Probing Mechanism/figrim/Targets",
    patch_tensor=adv_patch,
    transform=transform,
    device=DEVICE
)

# Stratify by adv_flag (clean vs patched)
y = np.array(train_dataset.targets)
idx = np.arange(len(y))

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(sss.split(idx, y))

train_dataset_split = Subset(train_dataset, train_idx.tolist())
test_dataset_split  = Subset(train_dataset, test_idx.tolist())

train_loader = DataLoader(train_dataset_split, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_dataset_split,  batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

# Example batch
images, class_labels, adv_flags = next(iter(train_loader))
print(images.shape, class_labels[:5], adv_flags[:5])


### Probing Network 

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image
import os




# ===== DETECTOR MODULE =====
class ProbingDetector(nn.Module):
    def __init__(self, base_model, layer_names, feature_dim=64):
        super().__init__()
        self.base = base_model
        self.layer_names = layer_names
        self.features = {}
        self.probes = nn.ModuleList()

        # Hook & MLP for each layer
        for name in layer_names:
            layer = dict([*self.base.named_modules()])[name]

            def get_hook(n):
                def hook(module, input, output):
                    self.features[n] = output.detach()
                return hook
            layer.register_forward_hook(get_hook(name))

            # Determine number of channels
            out_channels = layer.out_channels if hasattr(layer, 'out_channels') else 64
            self.probes.append(nn.Sequential(
                nn.AdaptiveAvgPool2d(1),
                nn.Flatten(),
                nn.Linear(out_channels, feature_dim),
                nn.ReLU()
            ))

        self.classifier = nn.Sequential(
            nn.Linear(len(layer_names) * feature_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        self.features = {name: None for name in self.layer_names}
        _ = self.base(x)

        processed = []
        for i, name in enumerate(self.layer_names):
            feat = self.features[name]
            if feat is not None:
                processed_feat = self.probes[i](feat)
                processed.append(processed_feat)

        concat = torch.cat(processed, dim=1)
        return self.classifier(concat)

In [ ]:
import torch.optim as optim
from tqdm import tqdm

# ===== Training Configuration =====
EPOCHS = 10
LR = 1e-3

# ===== CONFIGURATION =====
LAYER_NAMES = ['']  # For ResNet
LAYER_NAMES = [
    "layer1.0.conv1",   # early
    "layer2.0.conv1",   # mid
    "layer3.0.conv1",   # deeper
    "layer4.0.conv1"    # deepest before avgpool
]

def list_conv2d_names(m):
    return [name for name, mod in m.named_modules() if isinstance(mod, nn.Conv2d)]

conv_names = list_conv2d_names(model)
print(conv_names[:10], "... total:", len(conv_names))

LAYER_NAMES = [ # For EffNet
    "features.0.0",                # ✅ Early stem conv (3 → 32 channels, stride=2)
    "features.3.0.block.1.0",      # ✅ Middle depthwise conv (5×5, stride=2, ~24→40 channels)
    "features.5.1.block.1.0",      # ✅ Deeper depthwise conv (5×5, ~112→672 channels)
    "features.8.0"                 # ✅ Deepest 1×1 conv before classifier (320 → 1280)
]


IMAGE_SIZE = (224, 224)
FEATURE_DIM = 64  # Size of each layer’s embedding
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

detector = ProbingDetector(model, LAYER_NAMES, feature_dim=FEATURE_DIM).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(detector.parameters(), lr=LR)

detector.train()

for epoch in range(EPOCHS):
    running_loss = correct = total = 0

    for images, class_labels, adv_flags in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images = images.to(DEVICE)
        labels = adv_flags.to(DEVICE).long()      # 0=clean, 1=patched

        optimizer.zero_grad()
        outputs = detector(images)                # logits shape [B, 2]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.4f} - Acc: {accuracy:.2f}%")


In [ ]:
import torch
from sklearn.metrics import classification_report

# ===== Assume you already have these =====
# - test_loader
# - base_model (your pretrained classifier)
# - LAYER_NAMES (list of layer names to hook)
# - FEATURE_DIM (dimensionality of each probe head)

# Wrap your base model with the probing detector

detector.eval()

# ===== Inference and Evaluation =====
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels, adv_flags in test_loader:
        images = images.to(DEVICE)
        labels = adv_flags.to(DEVICE)
        
        outputs = detector(images)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

# ===== Report =====
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["Benign", "Adversarial"]))

from sklearn.metrics import precision_recall_fscore_support

# Per-class (0=Benign, 1=Adversarial)
p_cls, r_cls, f1_cls, sup_cls = precision_recall_fscore_support(
    all_labels, all_preds, labels=[0, 1], average=None, zero_division=0
)
print(f"Benign (0):       Precision={p_cls[0]:.4f}  Recall={r_cls[0]:.4f}  F1={f1_cls[0]:.4f}  Support={sup_cls[0]}")
print(f"Adversarial (1):  Precision={p_cls[1]:.4f}  Recall={r_cls[1]:.4f}  F1={f1_cls[1]:.4f}  Support={sup_cls[1]}")

# Binary score treating class 1 (Adversarial) as the positive class
p_bin, r_bin, f1_bin, sup_bin = precision_recall_fscore_support(
    all_labels, all_preds, average="binary", pos_label=1, zero_division=0
)
print(f"\nPositive=Adversarial → Precision={p_bin:.4f}  Recall={r_bin:.4f}  F1={f1_bin:.4f}  Support={sup_bin}")

# Averages
for avg in ["macro", "weighted", "micro"]:
    p, r, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average=avg, zero_division=0
    )
    print(f"{avg.capitalize()} avg:   Precision={p:.4f}  Recall={r:.4f}  F1={f1:.4f}")


### Senti-Net

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
from collections import Counter

device = "cuda:1"
model = model.to(device)
# === 1. Grad-CAM Generator ===
def simple_grad_cam(model, image_tensor, target_class, target_layer):
    model.eval()
    gradients = []
    activations = []

    def forward_hook(module, input, output):
        activations.append(output)

    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0])

    handle_fwd = target_layer.register_forward_hook(forward_hook)
    handle_bwd = target_layer.register_backward_hook(backward_hook)

    output = model(image_tensor)
    model.zero_grad()
    loss = output[0, target_class]
    loss.backward(retain_graph=True)

    activ = activations[0].detach()
    grad = gradients[0].detach()
    weights = grad.mean(dim=(2, 3), keepdim=True)
    cam = (weights * activ).sum(dim=1, keepdim=True)
    cam = F.relu(cam)
    cam = F.interpolate(cam, size=image_tensor.shape[-2:], mode='bilinear', align_corners=False)
    cam = cam.squeeze()

    handle_fwd.remove()
    handle_bwd.remove()

    return cam

# === 2. Patch Overlay ===
def overlay_patch(base_image, mask):
    patch_color = torch.tensor([1.0, 0.0, 0.0], device=base_image.device).view(3, 1, 1)  # red patch
    return base_image * (1 - mask) + patch_color * mask

# === 3. SentiNet-style Defense ===

# === 3. SentiNet-style Defense (Detector adaptation) ===
def sentinet_defense(model, test_loader, benign_images, grad_cam_fn, overlay_fn, target_layer,
                     threshold=0.2, device='cuda:1'):
    model.eval()

    stats_benign = []

    print("==> Calibrating boundary with benign references...")
    for img in tqdm(benign_images, desc="Calibrating"):
        img = img.unsqueeze(0).to(device)
        pred_label = model(img).argmax(dim=1).item()
        cam = grad_cam_fn(model, img, pred_label, target_layer)
        mask = (cam > threshold * cam.max()).float()

        fooled_count, inert_conf = 0, []
        for ref_img in benign_images:
            ref = ref_img.unsqueeze(0).to(device)

            # overlay suspect mask into decoy
            adv = overlay_fn(ref, mask)

            # === STRICT FOOL CRITERION ===
            # count as fooled only if decoy is classified as the suspect's predicted class
            if model(adv).argmax(dim=1).item() == pred_label:
                fooled_count += 1

            # inert overlay (noise)
            noise_overlay = overlay_fn(ref, torch.randn_like(mask) * 0.01)
            conf = F.softmax(model(noise_overlay), dim=1).max().item()
            inert_conf.append(conf)

        stats_benign.append([fooled_count, np.mean(inert_conf)])

    # Fit one-class boundary only on benign references
    clf = OneClassSVM(gamma='scale', nu=0.1)
    clf.fit(stats_benign)

    y_true, y_pred = [], []

    print("==> Evaluating on test set...")
    for images, labels, adv_flags in test_loader:
        images = images.to(DEVICE)
        labels = adv_flags.to(DEVICE)

        for i in range(images.size(0)):
            img = images[i].unsqueeze(0)
            true_label = labels[i].item()
            pred_label = model(img).argmax(dim=1).item()

            cam = grad_cam_fn(model, img, pred_label, target_layer)
            mask = (cam > threshold * cam.max()).float()

            fooled_count, inert_conf = 0, []
            for ref_img in benign_images:
                ref = ref_img.unsqueeze(0).to(device)
                adv = overlay_fn(ref, mask)

                # === STRICT FOOL CRITERION ===
                if model(adv).argmax(dim=1).item() == pred_label:
                    fooled_count += 1

                noise_overlay = overlay_fn(ref, torch.randn_like(mask) * 0.01)
                conf = F.softmax(model(noise_overlay), dim=1).max().item()
                inert_conf.append(conf)

            test_point = [[fooled_count, np.mean(inert_conf)]]
            prediction = clf.predict(test_point)[0]  # -1 = adversarial, 1 = benign

            pred_binary = 1 if prediction == -1 else 0
            y_true.append(true_label)
            y_pred.append(pred_binary)

    # === Final Metrics ===
    acc = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

    print(f"\n✅ SentiNet-Style Detection Metrics (strict criterion):")
    print(f"Accuracy : {acc * 100:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1 Score : {f1:.4f}")

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:
# Select a convolutional layer from your model
target_layer = dict(model.named_modules())['features.8']  # For ResNet

# Choose 10 benign images (label 0) from test_dataset
benign_images = [img for img, label in test_dataset if label == 0][:10]


results = sentinet_defense(
    model=model,
    test_loader=test_loader,
    benign_images=benign_images,
    grad_cam_fn=simple_grad_cam,
    overlay_fn=overlay_patch,
    target_layer=target_layer,
    threshold=0.1, #lower the better - most optimal value 
    device=DEVICE
)

print("\nFinal Results:", results)

### X-Detect

In [ ]:
import torch
import torch.nn.functional as F
from collections import Counter
import torchvision.transforms as T
import numpy as np
from sklearn.metrics import classification_report
import cv2
from PIL import Image

# ---- CONFIG ----
DEVICE = 'cuda:1'
NUM_CLASSES = 2  # Benign / Adversarial
TRANSFORMS = [
    lambda x: T.GaussianBlur(kernel_size=5)(x),
    lambda x: x + torch.randn_like(x) * 0.05,  # Noise
    lambda x: T.ColorJitter(brightness=0.3)(x),
    lambda x: T.RandomHorizontalFlip(p=1.0)(x),
]

# ---- SPD Detector ----
def scene_processing_detector(image, model, threshold=0.6):
    with torch.no_grad():
        orig_pred = F.softmax(model(image.unsqueeze(0).to(DEVICE)), dim=1)[0]

        pred_changes = []
        for tf in TRANSFORMS:
            perturbed = tf(image.clone().cpu())
            perturbed_pred = F.softmax(model(perturbed.unsqueeze(0).to(DEVICE)), dim=1)[0]
            delta = torch.norm(orig_pred - perturbed_pred).item()
            pred_changes.append(delta)

        avg_change = np.mean(pred_changes)
        is_adv = avg_change > threshold
        return is_adv, avg_change

# ---- OED Detector (SIFT + KNN, Placeholder using raw pixels) ----
def object_extraction_detector(image, model, prototypes, knn_k=3):
    sift = cv2.SIFT_create()
    img_np = image.permute(1, 2, 0).cpu().detach().numpy()
    img_np = (img_np * 255).astype(np.uint8)
    kp1, des1 = sift.detectAndCompute(img_np, None)

    similarities = []
    for label, proto_des in prototypes:
        if des1 is None or proto_des is None:
            similarities.append((label, 0))
            continue
        matches = cv2.BFMatcher().knnMatch(des1, proto_des, k=2)
        good = [m for m, n in matches if m.distance < 0.75 * n.distance]
        similarities.append((label, len(good)))

    # KNN voting
    similarities.sort(key=lambda x: -x[1])
    votes = [lbl for lbl, _ in similarities[:knn_k]]
    predicted = Counter(votes).most_common(1)[0][0]

    # Get CNN model prediction
    with torch.no_grad():
        cnn_pred = model(image.unsqueeze(0).to(DEVICE)).argmax(dim=1).item()

    is_adv = (predicted != cnn_pred)
    return is_adv, predicted, cnn_pred

# ---- Evaluation ----
def run_xdetect_on_testset(model, test_loader, prototypes):
    y_true, y_pred = [], []

    for images, labels, adv_flags in test_loader:
        for i in range(images.size(0)):
            image, label = images[i].to(DEVICE), adv_flags[i].item()

            # Run both detectors
            spd_flag, _ = scene_processing_detector(image, model)
            oed_flag, _, _ = object_extraction_detector(image, model, prototypes)

            final_prediction = int(spd_flag or oed_flag)  # 1 = Adversarial, 0 = Benign

            y_pred.append(final_prediction)
            y_true.append(label)

    print("=== X-Detect Classification Report ===")
    print(classification_report(y_true, y_pred, target_names=["Benign", "Adversarial"]))


In [ ]:
import cv2
from tqdm import tqdm

def extract_sift_prototypes(dataset, num_per_class=5):
    sift = cv2.SIFT_create()
    prototypes = []

    count = 0
    for img, label in dataset:
        if label == 0:  # Benign class only
            img_np = img.permute(1, 2, 0).numpy()
            img_np = (img_np * 255).astype(np.uint8)
            kp, des = sift.detectAndCompute(img_np, None)
            if des is not None:
                prototypes.append((label, des))
                count += 1
        if count >= num_per_class:
            break

    print(f"Extracted {count} SIFT prototypes.")
    return prototypes

# Example:
prototypes = extract_sift_prototypes(test_dataset, num_per_class=10)
results = run_xdetect_on_testset(model, test_loader, prototypes)

In [ ]:
import torch
import torch.nn.functional as F

def extract_deep_prototypes(model, dataset, target_layer, device, num=10):
    model.eval()
    features = []

    def hook(module, input, output):
        features.append(output.mean(dim=(2, 3)).squeeze().cpu())

    handle = target_layer.register_forward_hook(hook)

    prototypes = []
    count = 0
    for img, label in dataset:
        if label == 0:  # Only benign samples
            features.clear()
            model(img.unsqueeze(0).to(device))
            if features:
                prototypes.append((label, F.normalize(features[0], dim=0)))
                count += 1
        if count >= num:
            break

    handle.remove()
    print(f"Extracted {count} deep prototypes.")
    return prototypes


target_layer = dict(model.named_modules())['features.8']
prototypes = extract_deep_prototypes(model, test_dataset, target_layer, DEVICE, num=10)


from torch.nn.functional import cosine_similarity

def deep_feature_oed(model, image, target_layer, prototypes):
    model.eval()
    features = []

    def hook(module, input, output):
        features.append(output.mean(dim=(2, 3)).squeeze())

    handle = target_layer.register_forward_hook(hook)
    _ = model(image.unsqueeze(0))
    handle.remove()

    test_feat = F.normalize(features[0], dim=0)

    similarities = [
        cosine_similarity(test_feat, proto_feat.to(test_feat.device), dim=0).item()
        for _, proto_feat in prototypes
    ]

    max_sim = max(similarities)
    predicted_match = similarities.index(max_sim)

    cnn_pred = model(image.unsqueeze(0)).argmax(dim=1).item()
    is_adv = max_sim < 0.85  # threshold — tune if needed

    return is_adv, predicted_match, cnn_pred



def run_xdetect_on_testset(model, test_loader, prototypes, target_layer):
    y_true, y_pred = [], []

    for images, labels, adv_flags in test_loader:
        for i in range(images.size(0)):
            img = images[i].to(DEVICE)
            true_label = adv_flags[i].item()

            spd_flag, _ = scene_processing_detector(img, model)
            oed_flag, _, _ = deep_feature_oed(model, img, target_layer, prototypes)

            predicted = int(spd_flag or oed_flag)
            y_true.append(true_label)
            y_pred.append(predicted)

    from sklearn.metrics import classification_report
    print("=== X-Detect (with Deep OED) Report ===")
    print(classification_report(y_true, y_pred, target_names=["Benign", "Adversarial"]))


target_layer = dict(model.named_modules())['features.8']
prototypes = extract_deep_prototypes(model, test_dataset, target_layer, DEVICE, num=10)

# better performing 
run_xdetect_on_testset(model, test_loader, prototypes, target_layer)
